# *MODELO FINAL*

## 1. Selección del Mejor Modelo
Tras evaluar distintos modelos (KNN, Árboles de Decisión, Regresión Logística, SVM, etc) se comprobó que el SVM con Kernel RBF y los hiperparámetros óptimos:
* C=10
* gamma = 'scale'
* kernel = 'rbf'

presenta el mejor desempeño en términos de Balanced Accuracy en el conjunto de validación interna. Esto lo convierte en nuestro candidato para el modelo final

## 2. Evaluación Outer y Estimación del Desempeño futuro
La estrategia de evaluación outer consiste en separar los datos en dos partes:

* Train (2/3): Para realizar el ajuste y selección de modelos
* Test (1/3): Para obtener una estimación real de rendimiento que tendría el modelo en una competición

Aunque durante la práctica se ha usado esta partición para evaluar el desempeño, en el paso final se reentrena al modelo usando todos los datos disponibles. 

## 3.Entrenamiento final y Guardado del Modelo
En este paso se reentrena al modelo SVM Final utilizando todos los datos. Se aplica un preprocesado antes de entenar el clasificador. Finalmente guardamos el modelo en modelo_final.pkl

In [2]:
# Importamos las librerías necesarias
import pandas as pd
import pickle
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score
# ---------------------------
# 3.1. Cargar y Preprocesar los Datos
# ---------------------------
# Cargamos los datos disponibles de entrenamiento y el dataset de competición
datos_generales = pd.read_csv('./attrition_availabledata_10.csv.gz')
datos_test = pd.read_csv('./attrition_competition_10.csv.gz')  # Datos para el test de la competición

# Convertimos la variable objetivo a numérica: 'Yes' -> 1, 'No' -> 0
datos_generales['Attrition'] = datos_generales['Attrition'].map({'Yes': 1, 'No': 0})

# Eliminamos columnas que no aportan información (IDs, columnas constantes, etc.)
X = datos_generales.drop(columns=['Attrition', 'EmployeeID', 'Over18', 'EmployeeCount', 'StandardHours'])
y = datos_generales['Attrition']

# Definimos los nombres de variables según su tipo (estas definiciones se han usado en el preprocesado)
categorical_features = ['Department', 'JobRole', 'EducationField']
ordinal_features = ['BusinessTravel', 'Gender', 'MaritalStatus']
# Nota: el resto de las variables numéricas se mantienen sin transformación adicional
valores_ordinales = [
    ['Non-Travel', 'Travel_Rarely', 'Travel_Frequently'],
    ['Male', 'Female'],
    ['Single', 'Married', 'Divorced']
]

# Creamos el preprocesador para transformar variables categóricas y ordinales
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_features),  # Variables categóricas sin orden
        ('ord', OrdinalEncoder(categories=valores_ordinales), ordinal_features)  # Variables ordinales
    ],
    remainder='passthrough'  # El resto de variables (numéricas) se mantienen
)

# Aplicamos el preprocesado a todo el dataset de entrenamiento
X_preprocessed = preprocessor.fit_transform(X)
# Obtenemos nombres de columnas tras la transformación para posibles análisis posteriores
columnas_preprocesadas = preprocessor.get_feature_names_out()

# Convertimos el resultado a DataFrame (opcional, para visualización)
X_preprocessed = pd.DataFrame(X_preprocessed, columns=columnas_preprocesadas)

# ---------------------------
# 3.2. Entrenamiento del Modelo Final
# ---------------------------
# Definimos el pipeline final: imputación (por si acaso), escalado y SVM con hiperparámetros óptimos
final_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', RobustScaler()),
    ('svm', SVC(C=10, gamma='scale', kernel='rbf', probability=True))  # Se incluye probability=True si se requieren probabilidades
])

# Entrenamos el modelo final con la totalidad de los datos preprocesados
final_pipeline.fit(X_preprocessed, y)

# Realizamos predicciones en el conjunto de entrenamiento
y_pred = final_pipeline.predict(X_preprocessed)

# Calculamos la balanced accuracy
balanced_accuracy = balanced_accuracy_score(y, y_pred)
print(f"Balanced Accuracy en el conjunto de entrenamiento: {balanced_accuracy:.4f}")

# Guardamos el modelo final en un fichero 'modelo_final.pkl'
with open('modelo_final.pkl', 'wb') as f:
    pickle.dump(final_pipeline, f)

print("Modelo final guardado en 'modelo_final.pkl'")


Balanced Accuracy en el conjunto de entrenamiento: 0.9979
Modelo final guardado en 'modelo_final.pkl'


## 4.Generación de Predicciones para la Competición
El siguiente paso es utilizar el modelo final para generar predicciones sobre el conjunto de datos de la competición. Para ello se debe aplicar el mismo preprocesado que a los datos de entrenamiento. El fichero resultante se guardará como predicciones.csv

In [3]:
# ---------------------------
# 4.1. Preprocesamiento de los Datos de la Competición
# ---------------------------
# Para el conjunto de competición, eliminamos las mismas columnas que en el entrenamiento.
# Se asume que el dataset de competición no contiene la variable 'Attrition'
X_competicion = datos_test.drop(columns=['EmployeeID', 'Over18', 'EmployeeCount', 'StandardHours'])

# Aplicamos el preprocesado usando el preprocessor ya ajustado (fit) en el conjunto de entrenamiento
X_competicion_preprocessed = preprocessor.transform(X_competicion)
# Convertimos a DataFrame para mayor claridad
X_competicion_preprocessed = pd.DataFrame(X_competicion_preprocessed, columns=columnas_preprocesadas)

# ---------------------------
# 4.2. Generar Predicciones
# ---------------------------
# Utilizamos el modelo final para predecir las clases (o probabilidades si se requiere)
predicciones = final_pipeline.predict(X_competicion_preprocessed)

# Si se desea guardar además las probabilidades, se puede obtener:
# prob_pred = final_pipeline.predict_proba(X_competicion_preprocessed)

# Creamos un DataFrame con las predicciones
df_predicciones = pd.DataFrame({
    'Predicted_Attrition': predicciones
})

# Guardamos el DataFrame en un fichero CSV
df_predicciones.to_csv('predicciones.csv', index=False)

print("Predicciones guardadas en 'predicciones.csv'")


Predicciones guardadas en 'predicciones.csv'


In [4]:

import shap
import numpy as np
import pandas as pd
import warnings


X_background = shap.sample(X_preprocessed, 1, random_state=42)  # Toma 100 instancias al azar


explainer = shap.KernelExplainer(
    lambda x: final_pipeline.predict_proba(x),
    X_background,
    link="logit"
)

warnings.filterwarnings("ignore", category=UserWarning)

shap_values = explainer.shap_values(X_preprocessed)

shap.summary_plot(shap_values[1], X_preprocessed)



ModuleNotFoundError: No module named 'shap'